In [1]:

import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import urllib.request


# Cargar los datos
train_df = pd.read_csv(r'C:\Users\alber\OneDrive\Documentos\GitHub\DS_ONLINE_THEBRIDGE_ATC\Masterclass\Kaggle\data\train.csv', index_col="laptop_ID")
test_df = pd.read_csv(r'C:\Users\alber\OneDrive\Documentos\GitHub\DS_ONLINE_THEBRIDGE_ATC\Masterclass\Kaggle\data\test.csv', index_col="laptop_ID")

# Limpiar la columna Weight (eliminar 'kg' y convertir a float)
def clean_weight(value):
    return float(re.sub("[^0-9.]", "", value))

train_df["Weight"] = train_df["Weight"].apply(clean_weight)
test_df["Weight"] = test_df["Weight"].apply(clean_weight)

# Definir X e y
X = train_df.drop(columns=['Price_in_euros'])
y = train_df['Price_in_euros'].copy()

# Dividir datos en train y test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar variables numéricas y categóricas
numeric_features = ["Inches", "Weight"]
categorical_features = ["Company", "TypeName", "ScreenResolution", "Cpu", "Ram", "Memory", "Gpu", "OpSys"]

# Transformaciones
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
])

# Modelo optimizado
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(n_estimators=300, max_depth=15, random_state=42))
])

# Entrenamiento
model.fit(X_train, y_train)

# Evaluación
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE en validación: {rmse}")

# Predicción sobre test.csv
X_test = test_df.copy()
predictions = model.predict(X_test)

# Generar archivo de submission
submission = pd.DataFrame({
    "laptop_ID": X_test.index,
    "Price_euros": predictions
})


RMSE en validación: 354.0528680070684


In [13]:
sample = pd.DataFrame({"laptop_ID": X_test.index, "Price_in_euros": predictions})

In [8]:
submission = pd.DataFrame({"laptop_ID": X_test.index, "Price_in_euros": predictions})

In [9]:
submission.head()

,laptop_ID,Price_in_euros
0,209,1818.161916
1,1281,306.086418
2,1168,383.878043
3,1231,995.600258
4,1020,797.081505


In [10]:
submission.shape

(391, 2)

In [11]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission3.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [14]:
chequeador(submission)

You're ready to submit!


NameError: name 'urllib' is not defined